In [ ]:
import joblib
import numpy as np
import pandas as pd
import pickle
import pymysql

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from preprocessing.imbalanced_label import BinaryImbClsRS

In [ ]:
df = pd.read_csv("../../example_data/CW_MEDIA_CLICKVIEW_LOG_CTR_데이터_추출1.csv")

In [ ]:
col_order = ["createdDate", "mediaId", "inventoryId", "adverId", "adType", "frameId", "auid", "logType", # ordered
             #not ordered for now 
             "adProduct", "adCampain", "productCode", "cpoint", "mpoint",
             'remoteIp', 'platform', 'device', 'browser', 'freqLog',
             'tTime', 'kno', 'kwrdSeq', 'gender', 'age', 'osCode', 'price',
             'frameCombiKey'
            ]
df = df[col_order]

## EDA

In [ ]:
# df.groupby(["logType"]).count()

In [ ]:
# (df.nunique()
#    .to_frame()
#    .rename(columns={0:"# unique values"})
#    .T
# )

In [ ]:
# # When logType=V : W,M
# #              C : '7|14', '12|57', etc...
# df["gender"].unique()

In [ ]:
# # When logType=V : v, u, t, s, etc...
# #              C : '7|14', '12|57', etc...
# df.loc[df["logType"] == "V"]["age"].unique()

In [ ]:
# (df.loc[(df["mediaId"] == "mmediaseoul") &
#        (df["inventoryId"] == 18232) &
#        (df["adverId"] == "reebok1")]
#    .loc["2021-11-07 11:00:00":"2021-11-07 11:11:00"])

## Preprocess

In [ ]:
# use remoteIp to get city or town, etc...
# df["city"]

### Missing value imputation

In [ ]:
# # Column별 Missing Value 비율
# round(df.isna().sum()/len(df)*100, 2)

In [ ]:
df.replace("NONE", np.nan, inplace=True)
df.dropna(subset=["gender", "age"],inplace=True)

### Date related

In [ ]:
def is_weekday(x):
    """1 = weekday"""
    if x <= 4:
        return 1
    else:
        return 0
    
df["createdDate"] = pd.to_datetime(df["createdDate"])
df["year"] = df["createdDate"].dt.year
df["month"] = df["createdDate"].dt.month.astype(np.int8)
df["day"] = df["createdDate"].dt.day.astype(np.int8)

df["weekday"] = df["createdDate"].dt.dayofweek.apply(is_weekday).astype(np.int8)

# df.set_index(["createdDate"], inplace=True)

### Drop columns

In [ ]:
df["click"] = df["logType"].replace({"V":0, "C":1})

In [ ]:
df.drop(columns=[
    "frameId", # np.NaN 처리 하기전까지만 drop
    "adType", # df CW 만 사용하기때문
    "auid", # V-C Join 할때만 사용.
    "browser",
    "logType",
    "remoteIp",
    "device", #platform 이랑 동일
    "price", # 전부 0
    "kno", # 전부 0
    "kwrdSeq", # 전부 0, np.NaN
    "frameCombiKey", # 전부 np.NaN
    "createdDate", # y,m,d,weekday 변환되었음
], inplace=True)

In [ ]:
# df.groupby(["click"]).count()

In [ ]:
# # Column별 Missing Value 비율
# round(df.isna().sum()/len(df)*100, 2)

In [ ]:
# (df.nunique()
#    .to_frame()
#    .rename(columns={0:"# unique values"})
#    .T
#    .iloc[:, 4:]
# )

## forward-selection
: <i>정확히 파악한 column들만 사용하여 Model 을 만든다. 그후 파악되면 하나하나 추가하여 feature를 늘린다</i>

In [ ]:
cate_ord_cols = ["mediaId", "inventoryId", "adverId"]
cate_ohe_cols = ["platform"]
num_cols = ["cpoint", "mpoint", "freqLog", "tTime"]
others = ["year", "month", "day", "weekday"]
target = ["click"]

use_cols = cate_ord_cols + cate_ohe_cols + num_cols + others + target
df = df[use_cols]

In [ ]:
y = df["click"]
X = df.drop(columns=["click"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [ ]:
col_transformers = make_column_transformer(
                    (OneHotEncoder(), cate_ohe_cols),
                    (OrdinalEncoder(), cate_ord_cols),
                    (StandardScaler(), num_cols),
                    remainder='passthrough')

X_train_r = col_transformers.fit_transform(X_train)
X_test_r = col_transformers.fit_transform(X_test)

하지만 아래 confusion matrix 를 보면, 모두 `0 (No Click)` 으로 예측하는것으로 보임.

In [ ]:
clf = LogisticRegression(penalty="l2",
                         max_iter=1000)
clf.fit(X_train_r, y_train)

y_pred = clf.predict(X_test_r)
acc_sc = accuracy_score(y_test, y_pred)
print(f"Accuracy score = {round(acc_sc, 3)}")

plot_confusion_matrix(clf, X_test_r, y_test) 

### UnderSampling

In [ ]:
target_col = "click"
train_df = pd.concat([X_train, y_train], axis=1)
imb_pp = BinaryImbClsRS(train_df, target_col, True)

In [ ]:
df_us = imb_pp.random_undersample(-1)

In [ ]:
y_train_us = df_us[target_col]
X_train_us = df_us.drop(columns=[target_col])

X_train_us = col_transformers.fit_transform(X_train_us)
X_test = col_transformers.fit_transform(X_test)

In [ ]:
clf = LogisticRegression(penalty="l2",
                         max_iter=1000)

clf.fit(X_train_us, y_train_us)
y_pred = clf.predict(X_test)
acc_sc = accuracy_score(y_test, y_pred)
print(f"Accuracy score = {round(acc_sc, 3)}")

plot_confusion_matrix(clf, X_test, y_test) 

### OverSampling

In [ ]:
df_os = imb_pp.random_oversample(-1)
y_train_os = df_os[target_col]
X_train_os = df_os.drop(columns=[target_col])

X_train_os = col_transformers.fit_transform(X_train_os)

In [ ]:
clf = LogisticRegression(penalty="l2",
                         max_iter=1000)

clf.fit(X_train_os, y_train_os)
y_pred = clf.predict(X_test)
acc_sc = accuracy_score(y_test, y_pred)
print(f"Accuracy score = {round(acc_sc, 3)}")

plot_confusion_matrix(clf, X_test, y_test) 

### 저장된 transformer, 모델 로 예측

In [ ]:
# Prediction w/ saved model
transformer = joblib.load("trained_data/fitted_transformer.pkl") 
binary_clf = joblib.load("trained_data/us_fit_model.pkl")

X_test_tf = transformer.transform(X_test)
y_pred = binary_clf.predict(X_test_tf)
acc_sc = accuracy_score(y_test, y_pred)
print(f"Accuracy score = {round(acc_sc, 3)}")

plot_confusion_matrix(binary_clf, X_test_tf, y_test) 

### JSON input to prediction

In [ ]:
mediaId = "bangab"
inventoryId = "585494"
adverId = "nutricore"
platform = "W"
cpoint = 0.0
mpoint = 0.0
freqLog = 29
tTime = 51
year = 2021
month = 11
day = 11
weekday = 0


inp_data = {
    "mediaId":mediaId,
    "inventoryId":inventoryId,
    "adverId":adverId,
    "platform":platform,
    "cpoint":cpoint,
    "mpoint":mpoint,
    "freqLog":freqLog,
    "tTime":tTime,
    "year":year,
    "month":month,
    "day":day,
    "weekday":day
}

In [ ]:
pd.DataFrame({key:[item] for key, item in inp_data.items()})

In [ ]:
inp_data